In [37]:
import pandas as pd
import numpy as np
import autokeras as ak
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [38]:
df = pd.read_csv('TurinAHU.csv')
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Extracting day, month, year, and time into separate columns
df["day"] = df["Timestamp"].dt.day
df["month"] = df["Timestamp"].dt.month
df["year"] = df["Timestamp"].dt.year
df["time"] = df["Timestamp"].dt.hour * 3600 + df["Timestamp"].dt.minute * 60 + df["Timestamp"].dt.second

# Dropping the "timestamp" column
df = df.drop("Timestamp", axis=1)

# Reordering the columns
cols = df.columns.tolist()
cols = ["time", "day", "month", "year"] + cols[:-4]
df = df[cols]
df

,time,day,month,year,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power
0,79200,14,10,2019,19.859999,20.469999,18.5,19.020000,20.299999,71.110001,58.919998,79.5,0.0,0.0
1,80100,14,10,2019,19.855000,20.430000,18.5,19.020000,20.299999,71.320000,59.000000,82.0,0.0,0.0
2,81000,14,10,2019,19.850000,20.410000,18.5,19.020000,20.299999,71.470001,59.109997,79.5,0.0,0.0
3,81900,14,10,2019,19.840000,20.379999,18.5,19.080000,20.299999,71.439995,59.309998,77.0,0.0,0.0
4,82800,14,10,2019,19.830000,20.350000,18.5,19.080000,20.299999,71.580002,59.559998,79.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33883,74700,14,4,2021,19.539999,20.004999,20.5,19.619999,14.700000,39.020000,27.930000,57.0,0.0,0.0
33884,75600,14,4,2021,19.520000,19.949999,20.5,19.539999,13.700000,39.020000,28.090000,57.0,0.0,0.0
33885,76500,14,4,2021,19.430000,19.955000,20.5,19.420000,13.700000,39.399998,27.930000,57.0,0.0,0.0
33886,77400,14,4,2021,19.420000,19.920000,20.5,19.400000,13.700000,39.599998,28.039999,57.0,0.0,0.0


In [39]:
# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

X = df[['time', 'day', 'month', 'year']]
y = df.drop(columns=['time', 'day', 'month', 'year'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
# Initialize and train the AutoKeras model
regressor = ak.StructuredDataRegressor(max_trials=10, overwrite=True)
regressor.fit(X_train, y_train, verbose=2)

# Evaluate the model on the testing set
mse = mean_squared_error(y_test, regressor.predict(X_test))
mae = mean_absolute_error(y_test, regressor.predict(X_test))
mape = mean_absolute_percentage_error(y_test, regressor.predict(X_test))
rmse = np.sqrt(mse)

print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('Mean Absolute Percentage Error:', mape)
print('Root Mean Squared Error:', rmse)

# Make predictions
predictions = regressor.predict(X_test)

# Get predictions for the next 2 timesteps
next_timestep_predictions = regressor.predict(X_test, return_pred_sequences=True)[:, -2:, :]

# Print the predicted values for the next 2 timesteps
print('Predicted values for the next 2 timesteps:')
print(next_timestep_predictions)

AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations